### Quick notes about this notebook
Hi. I want to share my thoughts during this task as much as I can, that's why I decided to write down detailed notes as a go along. I trust it will be helpful.

### Importing libraries

In [57]:
import pandas as pd
import numpy as np

### Initial attempt to load data

In [58]:
dataset = pd.read_csv('unsd-citypopulation-year-fm_csv.txt')

### Initial exploratory
First, I will take a look on the size of the data, data completeness and column types

In [59]:
print(dataset.shape)
print(dataset.dtypes)
dataset.count()

(28528, 11)
Country or Area    object
Year               object
Area               object
Sex                object
City               object
City type          object
Record Type        object
Reliability        object
Source Year        object
Value              object
Value Footnotes    object
dtype: object


Country or Area    28528
Year               28528
Area               28365
Sex                28365
City               28365
City type          28365
Record Type        28365
Reliability        28365
Source Year        28365
Value              28365
Value Footnotes     9633
dtype: int64

It seens not a huge dataset. I got curious about a few "missing" records (28528 - 28365 = 163), then I tried the following command:

In [60]:
dataset[dataset['Area'].isnull()].head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
28364,footnoteSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28365,1,Including armed forces stationed in the area.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28366,2,Data refer to projections based on the 2001 Po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28367,3,The urban agglomeration of Buenos Aires includ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28368,4,The urban agglomeration of Tucumán-Tafí Viejo ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After that, just realised there are different headers on this dataset.

I will load the data into different datasets.

I have also seen this weird record, will take care. 

In [61]:
dataset.iloc[28377]

Country or Area                                                   13
Year               Locations are generally based on Statistical D...
Area                                                         Proper"
Sex                                                          usually
City                                                           based
City type                                                         on
Record Type                                                      the
Reliability                                               city inner
Source Year                                              Statistical
Value                                                          Local
Value Footnotes                                                Area"
Name: 28377, dtype: object

### Loading datasets properly

In [62]:
dataset1 = pd.read_csv('unsd-citypopulation-year-fm_csv.txt', nrows=28363, encoding='utf_8')
print(dataset1.count())
dataset1.head()

Country or Area    28363
Year               28363
Area               28363
Sex                28363
City               28363
City type          28363
Record Type        28363
Reliability        28363
Source Year        28363
Value              28363
Value Footnotes     9632
dtype: int64


,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445.0,NaN
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925.0,NaN
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408.0,NaN
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,NaN
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,NaN


In [63]:
# Load dataset 2, but skip one malformed record
dataset2 = pd.read_csv('unsd-citypopulation-year-fm_csv.txt', skiprows=list(range(0,28365)) + [28378], encoding='utf_8')
dataset2 = dataset2[['footnoteSeqID', 'Footnote']]

# Work around the malformed record 
Footnote = list(dataset.iloc[28377])[1:]
Footnote = ' '.join(Footnote)
footnoteSeqID = int(list(dataset.iloc[28377])[0])
missing_record = pd.DataFrame(data={'footnoteSeqID': [footnoteSeqID], 'Footnote': [Footnote]})

# Add the missing record
dataset2 = pd.concat([dataset2, missing_record], axis=0)

# Check data
print(dataset2.count())
dataset2.head()

footnoteSeqID    163
Footnote         163
dtype: int64


,footnoteSeqID,Footnote
0,1,Including armed forces stationed in the area.
1,2,Data refer to projections based on the 2001 Po...
2,3,The urban agglomeration of Buenos Aires includ...
3,4,The urban agglomeration of Tucumán-Tafí Viejo ...
4,5,Data refer to private households only.


### Mapping Value Footnotes

In [64]:
# Create a dictionary of footnotes
dict_of_footnotes = dict(dataset2.values)
dict_of_footnotes[-1] = "No comment has been added for this entry"

# Replace null values
dataset1['Value Footnotes'] = np.where(dataset1['Value Footnotes'].isnull(), '-1', dataset1['Value Footnotes'])

# Separeta all Value Footnotes in a Array os strings
array_footnotes_aux = dataset1['Value Footnotes'].values

# Convert from array of strings to array of array
array_footnotes = [x.split(',') for x in array_footnotes_aux]

# Map from id => description
res = []
for v in array_footnotes:
    new = [dict_of_footnotes[int(i)] for i in v if int(i) in dict_of_footnotes.keys()]
    if new == []:
        new = ['There was a note, but it was not present in the lookup table {}'.format(v)]
    res = res + [new]
    
# Enrich dataset1
dataset1['notes'] = res

dataset1.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,notes
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445.0,-1,[No comment has been added for this entry]
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925.0,-1,[No comment has been added for this entry]
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408.0,-1,[No comment has been added for this entry]
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,-1,[No comment has been added for this entry]
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,-1,[No comment has been added for this entry]


### Some exploratory work

In [65]:
print(dataset1.columns)

dataset1['id'] = dataset1.index
aggregations = dict()
aggregations['id'] = 'count'
dataset1.groupby(['Record Type'], as_index=False).agg(aggregations)

Index(['Country or Area', 'Year', 'Area', 'Sex', 'City', 'City type',
       'Record Type', 'Reliability', 'Source Year', 'Value', 'Value Footnotes',
       'notes'],
      dtype='object')


,Record Type,id
0,Census - de facto - complete tabulation,3441
1,Census - de jure - complete tabulation,10004
2,Census - de jure - sample tabulation,144
3,Estimate - de facto,1578
4,Estimate - de jure,12902
5,Record type not defined/applicable,10
6,Sample survey - de facto,100
7,Sample survey - de jure,184


#### De jure vs de facto census
* de jure - according to the regular or legal residence
* de facto - according to the place where enumerated

### Data aggregation and preparation for viz

In [66]:
dataset1.head()

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,notes,id
0,Åland Islands,2013,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5445.0,-1,[No comment has been added for this entry],0
1,Åland Islands,2013,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2014,5925.0,-1,[No comment has been added for this entry],1
2,Åland Islands,2012,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5408.0,-1,[No comment has been added for this entry],2
3,Åland Islands,2012,Total,Female,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2013,5896.5,-1,[No comment has been added for this entry],3
4,Åland Islands,2011,Total,Male,MARIEHAMN,City proper,Estimate - de jure,"Final figure, complete",2012,5363.5,-1,[No comment has been added for this entry],4


#### First, I will transpose the column "Sex".

In [67]:
# Tramspose
pivot = pd.pivot_table(dataset1,index=['Country or Area', 'Year', 'Area', 'City', 'City type', 'Record Type', 'Reliability'], \
               columns=['Sex'], \
               values=['Value'], \
               aggfunc=[np.sum])

# Convert pivot view to flat schema
dataset_transposed = pd.DataFrame(pivot.to_records())

# Rename columns
dataset_transposed.columns = ['Country or Area', 'Year', 'Area', 'City', 'City type', 'Record Type', 'Reliability', 'Female', 'Male']
dataset_transposed.head()

,Country or Area,Year,Area,City,City type,Record Type,Reliability,Female,Male
0,Albania,2003,Total,TIRANA,City proper,Estimate - de facto,"Final figure, complete",198857.0,194006.0
1,Albania,2011,Total,Durrës,City proper,Census - de jure - complete tabulation,"Final figure, complete",56738.0,56511.0
2,Albania,2011,Total,TIRANA,City proper,Census - de jure - complete tabulation,"Final figure, complete",215256.0,203239.0
3,American Samoa,2000,Total,PAGO PAGO,City proper,Census - de jure - complete tabulation,"Final figure, complete",2192.0,2086.0
4,Andorra,2003,Total,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,"Final figure, complete",10543.0,10702.0


#### Quick validation

In [68]:
dataset1[dataset1['City'] == 'TIRANA']

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes,notes,id
17,Albania,2011,Total,Male,TIRANA,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,203239.0,-1,[No comment has been added for this entry],17
19,Albania,2011,Total,Female,TIRANA,City proper,Census - de jure - complete tabulation,"Final figure, complete",2013,215256.0,-1,[No comment has been added for this entry],19
20,Albania,2003,Total,Male,TIRANA,City proper,Estimate - de facto,"Final figure, complete",2004,194006.0,-1,[No comment has been added for this entry],20
21,Albania,2003,Total,Female,TIRANA,City proper,Estimate - de facto,"Final figure, complete",2004,198857.0,-1,[No comment has been added for this entry],21


In [69]:
dataset_transposed[dataset_transposed['City'] == 'TIRANA']

,Country or Area,Year,Area,City,City type,Record Type,Reliability,Female,Male
0,Albania,2003,Total,TIRANA,City proper,Estimate - de facto,"Final figure, complete",198857.0,194006.0
2,Albania,2011,Total,TIRANA,City proper,Census - de jure - complete tabulation,"Final figure, complete",215256.0,203239.0


#### Next, I will create period comparissons
Note: I will not compare YoY because most of times we dont have data for consecutive years. I will compare current census vs prior census (always looking to the same group to keep it fair)

In [70]:
# Create a new column to store the total number do people, regardless the sex
dataset_transposed['Total'] = dataset_transposed['Female'] + dataset_transposed['Male']

# Data is already aggregate. I will just reorder it
aggregations = dict()
aggregations['Total'] = 'sum'
aggregations['Female'] = 'sum'
aggregations['Male'] = 'sum'
df = dataset_transposed.groupby(['Country or Area', 'City', 'City type', 'Record Type', 'Year'], as_index = False) \
.agg(aggregations) \
.sort_values(['Country or Area', 'City', 'City type', 'Record Type', 'Year'], ascending=[True,True,True,True,True])

# Create a key to compute dense rank
df['key'] = df['Country or Area'] + ' | ' + df['City'] + ' | ' + df['City type'] + ' | ' + df['Record Type'] 

# Compute dense rank based on the key
df['rank'] = df.key.rank(method='dense').astype(int)

# Fetch the prior periods
df['prior_period_total'] = df['Total'].shift()
df['prior_period_female'] = df['Female'].shift()
df['prior_period_male'] = df['Male'].shift()

# Fetch the prior rank for validation
df['prior_rank'] = df['rank'].shift()

# Keep only the prior period from the same groups
df['prior_period_total'] = np.where(df['prior_rank'] == df['rank'], df['prior_period_total'], np.NaN)
df['prior_period_female'] = np.where(df['prior_rank'] == df['rank'], df['prior_period_female'], np.NaN)
df['prior_period_male'] = np.where(df['prior_rank'] == df['rank'], df['prior_period_male'], np.NaN)

# Adjust
df['diff_total'] = np.where( ((df['Total'] > 0) & (df['prior_period_total'] > 0)), ((df['Total'] / df['prior_period_total'])-1), np.NaN)
df['diff_female'] = np.where( ((df['Female'] > 0) & (df['prior_period_female'] > 0)), ((df['Female'] / df['prior_period_female'])-1), np.NaN)
df['diff_male'] = np.where( ((df['Male'] > 0) & (df['prior_period_male'] > 0)), ((df['Male'] / df['prior_period_male'])-1), np.NaN)

# Round
df['diff_total'] = df['diff_total'].round(2)
df['diff_total'] = df['diff_total'].round(2)
df['diff_total'] = df['diff_total'].round(2)

# Check results
df.head(20)

,Country or Area,City,City type,Record Type,Year,Total,Female,Male,key,rank,prior_period_total,prior_period_female,prior_period_male,prior_rank,diff_total,diff_female,diff_male
0,Albania,Durrës,City proper,Census - de jure - complete tabulation,2011,113249.0,56738.0,56511.0,Albania | Durrës | City proper | Census - de j...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,TIRANA,City proper,Census - de jure - complete tabulation,2011,418495.0,215256.0,203239.0,Albania | TIRANA | City proper | Census - de j...,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,Albania,TIRANA,City proper,Estimate - de facto,2003,392863.0,198857.0,194006.0,Albania | TIRANA | City proper | Estimate - de...,3,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,American Samoa,PAGO PAGO,City proper,Census - de jure - complete tabulation,2000,4278.0,2192.0,2086.0,American Samoa | PAGO PAGO | City proper | Cen...,4,NaN,NaN,NaN,3.0,NaN,NaN,NaN
4,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2003,21245.0,10543.0,10702.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN
5,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2005,23587.0,11718.0,11869.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,21245.0,10543.0,10702.0,5.0,0.11,0.111448,0.109045
6,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2006,24211.0,11944.0,12267.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,23587.0,11718.0,11869.0,5.0,0.03,0.019287,0.033533
7,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2007,24574.0,12148.0,12426.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,24211.0,11944.0,12267.0,5.0,0.01,0.017080,0.012962
8,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2008,24632.0,12195.0,12437.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,24574.0,12148.0,12426.0,5.0,0.00,0.003869,0.000885
9,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2009,24779.0,12240.0,12539.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,24632.0,12195.0,12437.0,5.0,0.01,0.003690,0.008201


#### Create % Female and % Male

In [71]:
df['perc_male'] = df['Male'] / df['Total']
df['perc_female'] = df['Female'] / df['Total']

#### Tabulate information about the last Census

In [72]:
df_last_census_max_population = df.sort_values(by = ['Country or Area', 'City', 'Year', 'Total'], ascending = [True, True, False, False]).groupby(['Country or Area', 'City'], as_index=False).first()
df_last_census_max_population = df_last_census_max_population[['Country or Area', 'City', 'Year', 'Record Type', 'Total', 'diff_total']]
df_last_census_max_population.columns = ['Country or Area', 'City', 'Year Last Census', 'Record Type Last Census', 'Total Last Census', 'Diff Total Last Census']
df_last_census_max_population.head()

,Country or Area,City,Year Last Census,Record Type Last Census,Total Last Census,Diff Total Last Census
0,Albania,Durrës,2011,Census - de jure - complete tabulation,113249.0,NaN
1,Albania,TIRANA,2011,Census - de jure - complete tabulation,418495.0,NaN
2,American Samoa,PAGO PAGO,2000,Census - de jure - complete tabulation,4278.0,NaN
3,Andorra,ANDORRA LA VELLA,2011,Estimate - de jure,22205.0,-0.06
4,Armenia,Gyumri (Leninakan),2011,Census - de jure - complete tabulation,121976.0,-0.19


In [73]:
df = pd.merge(df, df_last_census_max_population, on=['Country or Area', 'City'], how='left')

### Flagging some outliers

In [74]:
aggregations = dict()
aggregations['key'] = 'count'
aggregations['Total'] = ['mean', 'std']

population = df.groupby(['Country or Area', 'City type', 'Record Type', 'Year'], as_index=False).agg(aggregations)
population = pd.DataFrame(population.to_records())
population.columns = ['index', 'Country or Area', 'City type', 'Record Type', 'Year', 'count', 'mean', 'std']
population = population.drop(['index'], axis=1)
population = population[population['count'] >= 10]

df_outliers = pd.merge(df, population, on=['Country or Area', 'City type', 'Record Type', 'Year'], how='left')
df_outliers['zscore'] = (df_outliers['Total'] - df_outliers['mean']) / df_outliers['std']
df_outliers['is_outlier_upper'] = np.where(df_outliers['zscore'] >= 3, 'YES', 'NO')
df_outliers['is_outlier_lower'] = np.where(df_outliers['zscore'] <= -1, 'YES', 'NO')

### Country / City aggregation

In [75]:
aggregations = dict()
aggregations['Year'] = ['min', 'max', 'nunique', 'unique']
tabulation = df_outliers.groupby(['Country or Area', 'City'], as_index=False).agg(aggregations)
tabulation = pd.DataFrame(tabulation.to_records())
tabulation.columns = ['index', 'Country or Area', 'City', 'First Census', 'Last Census', 'Quantity of Census', 'Years of Census']
tabulation = tabulation.drop(['index'], axis = 1)
tabulation.head(5)

,Country or Area,City,First Census,Last Census,Quantity of Census,Years of Census
0,Albania,Durrës,2011,2011,1,[2011]
1,Albania,TIRANA,2003,2011,2,"[2011, 2003]"
2,American Samoa,PAGO PAGO,2000,2000,1,[2000]
3,Andorra,ANDORRA LA VELLA,2003,2011,8,"[2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011]"
4,Armenia,Gyumri (Leninakan),2001,2011,7,"[2001, 2011, 2004, 2006, 2007, 2008, 2009]"


In [76]:
df_city_aggr = pd.merge(df_outliers, tabulation, on=['Country or Area', 'City'], how='left')
df_city_aggr.head()

,Country or Area,City,City type,Record Type,Year,Total,Female,Male,key,rank,...,count,mean,std,zscore,is_outlier_upper,is_outlier_lower,First Census,Last Census,Quantity of Census,Years of Census
0,Albania,Durrës,City proper,Census - de jure - complete tabulation,2011,113249.0,56738.0,56511.0,Albania | Durrës | City proper | Census - de j...,1,...,NaN,NaN,NaN,NaN,NO,NO,2011,2011,1,[2011]
1,Albania,TIRANA,City proper,Census - de jure - complete tabulation,2011,418495.0,215256.0,203239.0,Albania | TIRANA | City proper | Census - de j...,2,...,NaN,NaN,NaN,NaN,NO,NO,2003,2011,2,"[2011, 2003]"
2,Albania,TIRANA,City proper,Estimate - de facto,2003,392863.0,198857.0,194006.0,Albania | TIRANA | City proper | Estimate - de...,3,...,NaN,NaN,NaN,NaN,NO,NO,2003,2011,2,"[2011, 2003]"
3,American Samoa,PAGO PAGO,City proper,Census - de jure - complete tabulation,2000,4278.0,2192.0,2086.0,American Samoa | PAGO PAGO | City proper | Cen...,4,...,NaN,NaN,NaN,NaN,NO,NO,2000,2000,1,[2000]
4,Andorra,ANDORRA LA VELLA,Urban agglomeration,Estimate - de jure,2003,21245.0,10543.0,10702.0,Andorra | ANDORRA LA VELLA | Urban agglomerati...,5,...,NaN,NaN,NaN,NaN,NO,NO,2003,2011,8,"[2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011]"


### Exporting data

In [101]:
def implode (v):
    return "|".join([str(i) for i in v])

df_city_aggr['Years of Census'] = df_city_aggr['Years of Census'].apply(implode)
df_city_aggr[['Years of Census']].head(5)

,Years of Census
0,2011
1,2011|2003
2,2011|2003
3,2000
4,2003|2005|2006|2007|2008|2009|2010|2011


In [102]:
df_city_aggr.to_csv('dataset_elsa.csv', encoding='utf_8', index=False)